In [1]:
import json
import pandas as pd
from sqlanalyzer import column_parser

In [2]:
with open('./sqlanalyzer/data.json', 'r') as f:
    query_dict = json.load(f)

In [3]:
print(json.dumps(query_dict, indent=2), '\n\n\n')

{
  "mapbox_accounts0": {
    "main": "SELECT id,\n       email AS email_accounts,\n       accountlevel\nFROM mapbox_customer_data.accounts\nWHERE dt = '{run_date}'\n  AND COALESCE(LOWER(email), 'NULL') NOT LIKE ('%@mapbox.com')\n  AND COALESCE(accountlevel, 'NULL') NOT IN ('staff',\n                                             'mapbox-no-bill-1')\n  AND id NOT IN ('streets-review-production') ), "
  },
  "mbx_verticals": {
    "no alias": "SELECT *,           max(dt) OVER (PARTITION BY 'dummy') AS max_dt    FROM analytics.mbx_account_verticals",
    "main": "SELECT mapbox_account_id,\n       vertical\nFROM\nWHERE dt = max_dt ), "
  },
  "mbx_sfdc_accounts": {
    "a": "sfdc.mapbox_accounts",
    "b.id": "AND b.dt = '{run_date}' AND a.salesforce_account_id =",
    "main": "SELECT a.*,\n       b.name AS sfdc_account_name\nFROM sfdc.mapbox_accounts a\nLEFT JOIN sfdc.accounts b ON a.dt = '{run_date}'\nWHERE a.dt = '{run_date}' ), "
  },
  "enterprise_exception": {
    "(sfdc_acct_id)),": 

In [ ]:
query_dict['mbx_verticals']

In [ ]:
query_dict['mbx_sfdc_accounts']

In [ ]:
query_dict['apa_deals0']

In [ ]:
query_dict['apa_deals']

## metastore 

In [4]:
db_fields_1 = pd.DataFrame({'db_table': 'mapbox_customer_data.accounts', 
            'all_columns': ['id', 'accountlevel', 'email', 'created', 'service_metadata_version', 'account', 'num_requests', 'dt', 'customerid']})


In [5]:
db_fields_2 = pd.DataFrame({'db_table': 'analytics.mbx_account_verticals', 
            'all_columns': ['mapbox_account_id', 'email', 'salesforce_account_id', 'vertical', 'vertical_source', 'dt']})


In [6]:
db_fields_3 = pd.DataFrame({'db_table': 'sfdc.mapbox_accounts', 
            'all_columns': ['mapbox_account_id', 'mapbox_account_salesforce_id', 'salesforce_account_id', 'dt']})


In [7]:
db_fields_4 = pd.DataFrame({'db_table': 'sfdc.accounts', 
            'all_columns': ['name', 'dt', 'account_health_c', 'billing_state']})


In [8]:
db_fields_5 = pd.DataFrame({'db_table': 'analytics.apa_deals',
                           'all_columns': ['mbx_acct_id', 'prd_start_date', 'prd_end_date', 'dt', 'stripe_cust_id']})


In [9]:
db_fields_6 = pd.DataFrame({'db_table': 'analytics.ref_calendar',
                           'all_columns': ['vdate']})


In [10]:
db_fields_7 = pd.DataFrame({'db_table': 'payments.stripe_invoices',
                           'all_columns': ['date', 'discount_id', 'customer_id', 'received_at', 'amount_due']})


In [11]:
db_fields_8 = pd.DataFrame({'db_table': 'payments.stripe_discounts',
                           'all_columns': ['id', 'coupon_id', 'subscription', 'received_at']})


In [12]:
db_fields_9 = pd.DataFrame({'db_table': 'payments.stripe_coupons',
                           'all_columns': ['id', 'duration', 'percent_off', 'created', 'currency']})


In [13]:
db_fields_10 = pd.DataFrame({'db_table': 'analytics.service_endpoint_mapping',
                           'all_columns': ['parent_sku', 'sku_id', 'service_org', 'dt', 'platform']})


In [14]:
db_fields_11 = pd.DataFrame({'db_table': 'analytics.api_requests',
                           'all_columns': ['log_source', 'platform', 'service', 'country',
                                          'status_code', 'request_type', 'plugin', 'account', 'hr',
                                          'num_requests', 'dt', 'hr_segment']})


In [15]:
db_fields_12 = pd.DataFrame({'db_table': 'analytics.service_endpoint_mapping',
                           'all_columns': ['parent_sku', 'sku_id', 'service_org', 'dt', 'platform']})


In [16]:
df = db_fields_1.append(db_fields_2, ignore_index=True)
df = df.append(db_fields_3, ignore_index=True)
df = df.append(db_fields_4, ignore_index=True)
df = df.append(db_fields_5, ignore_index=True)
df = df.append(db_fields_6, ignore_index=True)
df = df.append(db_fields_7, ignore_index=True)
df = df.append(db_fields_8, ignore_index=True)
df = df.append(db_fields_9, ignore_index=True)
df = df.append(db_fields_10, ignore_index=True)
df = df.append(db_fields_11, ignore_index=True)
df = df.append(db_fields_12, ignore_index=True)
df

,db_table,all_columns
0,mapbox_customer_data.accounts,id
1,mapbox_customer_data.accounts,accountlevel
2,mapbox_customer_data.accounts,email
3,mapbox_customer_data.accounts,created
4,mapbox_customer_data.accounts,service_metadata_version
...,...,...
60,analytics.service_endpoint_mapping,parent_sku
61,analytics.service_endpoint_mapping,sku_id
62,analytics.service_endpoint_mapping,service_org
63,analytics.service_endpoint_mapping,dt


In [17]:
def extract_subquery_fields(query, db_fields):
    formatter = column_parser.Parser(query)
    formatted = formatter.format_query(query)
    fields = formatter.match_queried_fields(formatted, db_fields)
    return fields


In [18]:
def compile_queried_cols(query_dict):
    all_cols = []
    for k,v in query_dict.items():
        if isinstance(v, dict):
            for k1,v1 in v.items():
                all_cols.extend(extract_subquery_fields(v1, df))
        else:
            all_cols.extend(extract_subquery_fields(v, df))
    return all_cols


In [19]:
for k,v in query_dict.items():
    if isinstance(v, dict):
        print(k, '\n', compile_queried_cols(v), '\n\n')
    else:
        print(k, '\n', extract_subquery_fields(v, df), '\n\n')

mapbox_accounts0 
 [{'database_name': 'mapbox_customer_data', 'table_name': 'accounts', 'column_name': 'dt'}, {'database_name': 'mapbox_customer_data', 'table_name': 'accounts', 'column_name': 'id'}, {'database_name': 'mapbox_customer_data', 'table_name': 'accounts', 'column_name': 'accountlevel'}, {'database_name': 'mapbox_customer_data', 'table_name': 'accounts', 'column_name': 'email'}] 


mbx_verticals 
 [{'database_name': 'analytics', 'table_name': 'mbx_account_verticals', 'column_name': 'dt'}] 


mbx_sfdc_accounts 
 [{'database_name': 'sfdc', 'table_name': 'mapbox_accounts', 'column_name': 'dt'}, {'database_name': 'sfdc', 'table_name': 'mapbox_accounts', 'column_name': 'mapbox_account_salesforce_id'}, {'database_name': 'sfdc', 'table_name': 'mapbox_accounts', 'column_name': 'mapbox_account_id'}, {'database_name': 'sfdc', 'table_name': 'accounts', 'column_name': 'name'}, {'database_name': 'sfdc', 'table_name': 'mapbox_accounts', 'column_name': 'salesforce_account_id'}] 


enterpri